# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [25]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [26]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

init_x0 (generic function with 1 method)

In [27]:
function Compute_AlphaAff_dual(v_k, dir_v_k)
    n = length(v_k)
    if n == 0
        return 0.
    end
    alphas = ones(n)
    for i=1:n
        if dir_v_k[i] < 1e-12
            alphas[i] = -v_k[i] * 0.95 / dir_v_k[i]
        end
    end
    alpha_min = minimum(alphas)
    return min(1, alpha_min)
end

function Compute_AlphaMax_dual(v_k, dir_v_k)
    n = length(v_k)
    if n == 0
        return 0.
    end
    alphas = ones(n)*100/99
    for i=1:n
        if dir_v_k[i] < 1e-12
            alphas[i] = -v_k[i] * 0.95 / dir_v_k[i]
        end
    end
    alpha_min = minimum(alphas)
    return alpha_min
end

function Compute_AlphaAff_primal(v_k, dir_v_k, lvar, uvar)
    n = length(v_k)
    alpha_l, alpha_u = ones(n), ones(n)
    for i=1:n
        if dir_v_k[i] > 1e-12
            alpha_u[i] = (uvar[i] - v_k[i]) * 0.95 / dir_v_k[i]
        elseif dir_v_k[i] < 1e-12
            alpha_l[i] = (lvar[i] - v_k[i]) * 0.95 / dir_v_k[i]
        end
    end
    alpha_min = min(minimum(alpha_l), minimum(alpha_u))
    return min(1, alpha_min)
end
    
function Compute_AlphaMax_primal(v_k, dir_v_k, lvar, uvar)
    n = length(v_k)
    alpha_l, alpha_u = ones(n)*100/99, ones(n)*100/99
    for i=1:n
        if dir_v_k[i] > 1e-12
            alpha_u[i] = (uvar[i] - v_k[i]) * 0.95 / dir_v_k[i]
        elseif dir_v_k[i] < 1e-12
            alpha_l[i] = (lvar[i] - v_k[i]) * 0.95 / dir_v_k[i]
        end
    end
    alpha_min = min(minimum(alpha_l), minimum(alpha_u))
    return alpha_min
end

function Compute_mu(x_l, x_u, s, stilde, lvar, uvar)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s' * (x_l-lvar) + stilde' * (uvar-x_u)) / (length(x_l) + length(x_u))
end

Compute_mu (generic function with 1 method)

In [28]:

function MehrotraPCQuadBounds(QM, max_iter; eps=1e-10, tol_step_x=1e-8, eps_mu=1e-8, display=true)
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    x_0 = init_x0(lvar, uvar)
    id_non_inf_lvar, id_non_inf_uvar = findall((x -> x!=-Inf), lvar), findall((x -> x!=Inf), uvar) # finite bounds index
    @assert all(x_0 .> lvar) && all(x_0 .< uvar)
    A = jac(QM, x_0)
    n_rows, n_cols = size(A) 
    Q = hess(QM, x_0)
    c = grad(QM, zeros(n_cols))
    c0 = obj(QM, zeros(n_cols))
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    s_0, stilde_0 = zeros(n_cols), zeros(n_cols)
    s_0[id_non_inf_lvar] = ones(length(id_non_inf_lvar))
    stilde_0[id_non_inf_uvar] = ones(length(id_non_inf_uvar))
    lambda_0 = Matrix(A)'\(c -s_0+stilde_0+Q*x_0) # least square initialisation, s_0 = stilde_0
    x_k, lambda_k, s_k, stilde_k = copy(x_0), copy(lambda_0), copy(s_0), copy(stilde_0)
    
    rb_0 = A * x_0 - b
    rc_0 = -Q * x_0 + A' * lambda_0 + s_0 - stilde_0 - c
    mu_0 = Compute_mu(x_0[id_non_inf_lvar], x_0[id_non_inf_uvar], 
                        s_0[id_non_inf_lvar], stilde_0[id_non_inf_uvar], 
                        lvar[id_non_inf_lvar], uvar[id_non_inf_uvar])

    # matrices without infinity constraints
    nb_non_inf_l, nb_non_inf_u = length(id_non_inf_lvar), length(id_non_inf_uvar) # number of finite constraints
    nb_inf_l, nb_inf_u = n_cols-nb_non_inf_l, n_cols-nb_non_inf_u # number of infinite constraints
    Xk_non_inf_l = Diagonal(x_k[id_non_inf_lvar])
    Sk_non_inf = [Diagonal(s_k[id_non_inf_lvar])  zeros(nb_non_inf_l, nb_inf_l)] # add zeros to match number of cols
    Xk_non_inf_u = Diagonal(x_k[id_non_inf_uvar])
    Stildek_non_inf = [Diagonal(stilde_k[id_non_inf_uvar])  zeros(nb_non_inf_u, nb_inf_u)] 
    Lvar_non_inf = Diagonal(lvar[id_non_inf_lvar]) 
    Uvar_non_inf = Diagonal(uvar[id_non_inf_uvar]) 
    
    mu_k, rb_k, rc_k = copy(mu_0), copy(rb_0), copy(rc_0)
    k = 0
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps # start false?
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    

    while k<max_iter && (opti_pdd==false || opti_rb==false || opti_rc==false) && small_step_x==false && small_mu==false
 
        # Affine scaling direction
        Jacob_Fk = [-Q                      A'                     I(n_cols)[:, 1:nb_non_inf_l]       I(n_cols)[:, 1:nb_non_inf_u]*(-1)
                    A                zeros(n_rows, n_rows)         zeros(n_rows, nb_non_inf_l)        zeros(n_rows, nb_non_inf_u)
                    Sk_non_inf       zeros(nb_non_inf_l, n_rows)    Xk_non_inf_l-Lvar_non_inf         zeros(nb_non_inf_l, nb_non_inf_u)
                    Stildek_non_inf  zeros(nb_non_inf_u, n_rows)   zeros(nb_non_inf_u,nb_non_inf_l)   Xk_non_inf_u-Uvar_non_inf]
        Fk_aff = [-rc_k
                  -rb_k
                  -(x_k[id_non_inf_lvar]-lvar[id_non_inf_lvar]).*s_k[id_non_inf_lvar]
                  -(x_k[id_non_inf_uvar]-uvar[id_non_inf_uvar]).*stilde_k[id_non_inf_uvar]]

        dir_aff_k = Jacob_Fk\Fk_aff
        
        alpha_aff_pri = Compute_AlphaAff_primal(x_k, dir_aff_k[1:n_cols], lvar, uvar)
        alpha_aff_dual = Compute_AlphaAff_dual(s_k[id_non_inf_lvar], 
                                            dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        alphatilde_aff_dual = Compute_AlphaAff_dual(stilde_k[id_non_inf_uvar], 
                                                dir_aff_k[n_rows+n_cols+nb_non_inf_l+1:end])
        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        if nb_non_inf_l == 0
            alpha_aff_dual_final = alphatilde_aff_dual
        elseif nb_non_inf_u == 0
            alpha_aff_dual_final = alpha_aff_dual
        else
            alpha_aff_dual_final = min(alpha_aff_dual, alphatilde_aff_dual)
        end
        
        mu_aff = Compute_mu(x_k[id_non_inf_lvar] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_lvar],
                    x_k[id_non_inf_uvar] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_uvar],
                    s_k[id_non_inf_lvar] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l],
                    stilde_k[id_non_inf_uvar] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end],
                    lvar[id_non_inf_lvar], uvar[id_non_inf_uvar])
        
        sigma = (mu_aff / mu_k)^3

        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e[1:nb_non_inf_l] - dir_aff_k[1:n_cols][id_non_inf_lvar].*dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
                 sigma*mu_k*e[1:nb_non_inf_u] - dir_aff_k[1:n_cols][id_non_inf_uvar].*dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end]] 
        
        dir_cc_k = Jacob_Fk\Fk_cc
        dir_k = dir_aff_k .+ dir_cc_k # final direction
        
        alpha_max_pri = Compute_AlphaMax_primal(x_k, dir_k[1:n_cols], lvar, uvar)
        alpha_max_dual = Compute_AlphaMax_dual(s_k[id_non_inf_lvar], 
                                        dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        
        alphatilde_max_dual = Compute_AlphaMax_dual(stilde_k[id_non_inf_uvar], 
                                            dir_k[n_rows+n_cols+nb_non_inf_l+1: end])
        
    
        # new parameters
        alpha_k_pri = min(0.99*alpha_max_pri, 1)
        alpha_k_dual = min(0.99*alpha_max_dual, 1)
        alphatilde_k_dual = min(0.99*alphatilde_max_dual, 1)
        if nb_non_inf_l == 0
            alpha_dual_final = alphatilde_k_dual
        elseif nb_non_inf_u == 0
            alpha_dual_final = alpha_k_dual
        else
            alpha_dual_final = min(alpha_k_dual, alphatilde_k_dual)
        end
        
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_dual_final * dir_k[n_cols+1: n_rows+n_cols]
        s_k[id_non_inf_lvar] += alpha_dual_final * dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
        stilde_k[id_non_inf_uvar] += alpha_dual_final * dir_k[n_rows+n_cols+nb_non_inf_l+1: end]
        
        Xk_non_inf_l = Diagonal(x_k[id_non_inf_lvar])
        Sk_non_inf = [Diagonal(s_k[id_non_inf_lvar])  zeros(nb_non_inf_l, nb_inf_l)] 
        Xk_non_inf_u = Diagonal(x_k[id_non_inf_uvar])
        Stildek_non_inf = [Diagonal(stilde_k[id_non_inf_uvar])  zeros(nb_non_inf_u, nb_inf_u)] 
        
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = Compute_mu(x_k[id_non_inf_lvar], x_k[id_non_inf_uvar], 
                        s_k[id_non_inf_lvar], stilde_k[id_non_inf_uvar], 
                        lvar[id_non_inf_lvar], uvar[id_non_inf_uvar])

        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k - stilde_k - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2)) # test correct? ecart primal dual
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        
        k += 1
        
        if display == true
            @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
    end
    
    if display == true
        criteria = [k >= max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, "stilde_opt" => stilde_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [29]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [30]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [31]:
res_mpc1 =  MehrotraPCQuadBounds(SM, 20);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -4.00e+00    |         1.20e+00       |    5.59e-01  |    1.72e-01  |  0.00e+00  |  1.00e+00
   1 |      9.46e+00    |         2.74e-02       |    1.01e-01  |    1.87e-01  |  2.11e+00  |  2.89e+00
   2 |      1.29e+01    |         2.78e-02       |    1.09e-02  |    1.91e-02  |  3.48e-01  |  1.88e-01
   3 |      1.35e+01    |         2.18e-03       |    7.33e-04  |    1.28e-03  |  4.59e-02  |  1.14e-02
   4 |      1.35e+01    |         1.31e-04       |    4.37e-05  |    7.61e-05  |  3.23e-03  |  6.79e-04
   5 |      1.35e+01    |         7.81e-06       |    2.61e-06  |    4.54e-06  |  1.92e-04  |  4.04e-05
   6 |      1.35e+01    |         4.68e-07       |    1.55e-07  |    2.71e-07  |  1.15e-05  |  2.40e-06
   7 |      1.35e+01    |         2.80e-08       |    9.27e-09  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [8.19429e-10, 3.0, 1.15934e-9]
  "lambda_opt" => [-9.46686, 12.0]
  "s_opt"      => [1.46686, 7.93551e-9, 0.466862]
  "stilde_opt" => [0.0, 0.0, 0.0]
  "n_iter"     => 8
  "pdd"        => 1.67496e-9
  "cond_rb"    => 5.53083e-10
  "cond_rc"    => 9.63727e-10

In [32]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
res_mpc_verif =  MehrotraPCQuadBounds(SM_verif, 20);
display_results(res_mpc_verif)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -4.00e+00    |         1.20e+00       |    5.59e-01  |    1.72e-01  |  0.00e+00  |  1.00e+00
   1 |      9.47e+00    |         2.79e-02       |    1.01e-01  |    1.87e-01  |  2.12e+00  |  2.88e+00
   2 |      1.29e+01    |         2.76e-02       |    1.09e-02  |    1.89e-02  |  3.48e-01  |  1.87e-01
   3 |      1.35e+01    |         2.19e-03       |    7.32e-04  |    1.28e-03  |  4.55e-02  |  1.13e-02
   4 |      1.35e+01    |         1.32e-04       |    4.37e-05  |    7.62e-05  |  3.22e-03  |  6.76e-04
   5 |      1.35e+01    |         7.90e-06       |    2.61e-06  |    4.55e-06  |  1.93e-04  |  4.02e-05
   6 |      1.35e+01    |         4.74e-07       |    1.56e-07  |    2.72e-07  |  1.15e-05  |  2.39e-06
   7 |      1.35e+01    |         2.85e-08       |    9.32e-09  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [-7.9316e-10, -3.0, -1.19842e-9]
  "lambda_opt" => [-9.4401, 12.0]
  "s_opt"      => [0.0, 0.0, 0.0]
  "stilde_opt" => [1.4401, 7.91363e-9, 0.440101]
  "n_iter"     => 8
  "pdd"        => 1.71093e-9
  "cond_rb"    => 5.56664e-10
  "cond_rc"    => 9.70106e-10

In [33]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
res_mpc_verif2 =  MehrotraPCQuadBounds(SM_verif2, 20);
display_results(res_mpc_verif2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.00e+00    |         8.33e-01       |    1.00e+00  |    4.08e-01  |  0.00e+00  |  1.00e+00
   1 |      1.11e-01    |         4.06e-01       |    0.00e+00  |    0.00e+00  |  1.19e+00  |  1.50e-01
   2 |      6.61e-03    |         2.67e-02       |    0.00e+00  |    0.00e+00  |  1.28e-01  |  8.95e-03
   3 |      3.93e-04    |         1.60e-03       |    0.00e+00  |    1.11e-16  |  7.62e-03  |  5.33e-04
   4 |      2.34e-05    |         9.51e-05       |    0.00e+00  |    1.53e-19  |  4.53e-04  |  3.17e-05
   5 |      1.39e-06    |         5.66e-06       |    0.00e+00  |    9.58e-21  |  2.70e-05  |  1.89e-06
   6 |      8.29e-08    |         3.37e-07       |    0.00e+00  |    5.99e-22  |  1.60e-06  |  1.12e-07
   7 |      4.93e-09    |         2.00e-08       |    0.00e+00  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [4.93014e-9, 0.5, 0.5]
  "lambda_opt" => [-1.51028e-8]
  "s_opt"      => [1.0, 1.51028e-8, 1.51028e-8]
  "stilde_opt" => [0.0, 0.0, 0.0]
  "n_iter"     => 7
  "pdd"        => 2.0033e-8
  "cond_rb"    => 0.0
  "cond_rc"    => 5.55112e-17

# Lecture des donnees .SIF

In [34]:

function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, x0=x0, name=name_pb)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

In [35]:
path = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes"
afiro = string(path, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes\\AFIRO.SIF"

In [36]:
qpdata2 = readqps(afiro)
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2, "QM2")
SM2 = SlackModel(QM2);
#displayQuadraticModel(SM2)

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [37]:
res_mpc2 =  MehrotraPCQuadBounds(SM2, 20, tol_step_x = 1e-10);
display_results(res_mpc2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      8.20e+00    |         1.13e+01       |    1.85e+01  |    7.29e-01  |  0.00e+00  |  1.00e+00
   1 |      6.91e+00    |         9.30e+00       |    1.84e+01  |    1.20e+00  |  3.51e+00  |  1.95e+00
   2 |      6.97e+00    |         9.71e+00       |    1.84e+01  |    1.39e+00  |  4.20e-01  |  2.18e+00
   3 |      6.96e+00    |         9.74e+00       |    1.84e+01  |    1.39e+00  |  1.31e-01  |  2.18e+00
   4 |      6.94e+00    |         9.77e+00       |    1.84e+01  |    1.39e+00  |  1.32e-01  |  2.18e+00
   5 |      6.91e+00    |         9.80e+00       |    1.84e+01  |    1.39e+00  |  1.33e-01  |  2.17e+00
   6 |      6.89e+00    |         9.82e+00       |    1.84e+01  |    1.39e+00  |  1.33e-01  |  2.17e+00
   7 |      6.87e+00    |         9.85e+00       |    1.84e+01  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [1.10484, 1.17541, 0.925871, 1.11135, 1.49529, 1.40662, 0.135…
  "lambda_opt" => [-1.57658, 0.545957, -1.26673, -0.334917, -2.43694, 1.25016, …
  "s_opt"      => [1.14553, 0.934263, 1.86132, 1.12072, 0.325871, 0.335802, 5.7…
  "stilde_opt" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.93731…
  "n_iter"     => 19
  "pdd"        => 9.96699
  "cond_rb"    => 18.4424
  "cond_rc"    => 1.38899

In [38]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3, "QM3")
SM3 = SlackModel(QM3);

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


In [39]:
res_mpc3 =  MehrotraPCQuadBounds(SM3, 20, tol_step_x = 1e-12);
display_results(res_mpc3)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.18e-01  |  0.00e+00  |  6.18e+00
   1 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.17e-01  |  1.37e+01  |  6.42e+00
   2 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.41e-01  |  3.47e-01  |  6.45e+00
   3 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.97e-01  |  2.00e-02  |  6.48e+00
   4 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    5.12e-01  |  2.19e-03  |  6.49e+00
   5 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    5.13e-01  |  1.72e-04  |  6.49e+00
   6 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    5.13e-01  |  1.05e-05  |  6.49e+00
   7 |     -1.01e+03    |         9.99e-01       |    3.11e+03  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [0.992826, 4.98986, 1.00405, 1.00575, 1.00379, 0.997181, 1.00…
  "lambda_opt" => [1.43718, 0.951314, 3.03567, -3.02319, 11.8686, 3.78856, 9.58…
  "s_opt"      => [1.22213, 1.04084, 1.08162, 0.862707, 1.14729, 1.13536, 0.784…
  "stilde_opt" => [0.0, 0.994065, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.…
  "n_iter"     => 12
  "pdd"        => 0.999012
  "cond_rb"    => 3112.11
  "cond_rc"    => 0.512584

In [40]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path, "\\SC50A.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4, "QM4")
SM4 = SlackModel(QM4);

┌ Info: Using 'SC50A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'MAXIM' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'CONST' as RHS (l. 153)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [41]:
res_mpc4 =  MehrotraPCQuadBounds(SM3, 20, tol_step_x = 1e-12);
display_results(res_mpc4)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.18e-01  |  0.00e+00  |  6.18e+00
   1 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.17e-01  |  1.37e+01  |  6.42e+00
   2 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.41e-01  |  3.47e-01  |  6.45e+00
   3 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    4.97e-01  |  2.00e-02  |  6.48e+00
   4 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    5.12e-01  |  2.19e-03  |  6.49e+00
   5 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    5.13e-01  |  1.72e-04  |  6.49e+00
   6 |     -1.01e+03    |         9.99e-01       |    3.11e+03  |    5.13e-01  |  1.05e-05  |  6.49e+00
   7 |     -1.01e+03    |         9.99e-01       |    3.11e+03  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [0.992826, 4.98986, 1.00405, 1.00575, 1.00379, 0.997181, 1.00…
  "lambda_opt" => [1.43718, 0.951314, 3.03567, -3.02319, 11.8686, 3.78856, 9.58…
  "s_opt"      => [1.22213, 1.04084, 1.08162, 0.862707, 1.14729, 1.13536, 0.784…
  "stilde_opt" => [0.0, 0.994065, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.…
  "n_iter"     => 12
  "pdd"        => 0.999012
  "cond_rb"    => 3112.11
  "cond_rc"    => 0.512584